In [1058]:
import numpy as np
import pandas as pd
from datetime import datetime

In [1059]:
from google.cloud import bigquery

client = bigquery.Client()

## Preprocess historical odds

In [1062]:
# read excel files 
df18 = pd.read_excel('eric_xia/raw_betting_odds/ncaa basketball 2018-19.xlsx') 
df19 = pd.read_excel('eric_xia/raw_betting_odds/ncaa basketball 2019-20.xlsx') 
df20 = pd.read_excel('eric_xia/raw_betting_odds/ncaa basketball 2020-21.xlsx') 
df21 = pd.read_excel('eric_xia/raw_betting_odds/ncaa basketball 2021-22.xlsx') 
df22 = pd.read_excel('eric_xia/raw_betting_odds/ncaa basketball 2022-23.xlsx') 

In [1063]:
# add season column 
df18['Season'] = '2018-19'
df19['Season'] = '2019-20'
df20['Season'] = '2020-21' 
df21['Season'] = '2021-22' 
df22['Season'] = '2022-23' 

In [1064]:
# Stack into one dataframe 
prev_odds = pd.concat([df18, df19, df20, df21, df22], ignore_index=True, axis=0)
prev_odds = prev_odds.drop(['Rot'], axis=1)
prev_odds

,Date,VH,Team,1st,2nd,Final,Open,Close,ML,2H,Season
0,1106,V,Towson,19.0,23.0,42.0,130,129,6500,67,2018-19
1,1106,H,Virginia,28.0,45.0,73.0,25,26,-16500,11,2018-19
2,1106,V,WiscMilwaukee,37.0,16.0,53.0,146,147.5,1025,78,2018-19
3,1106,H,BostonCollege,27.0,46.0,73.0,17,16,-1550,12.5,2018-19
4,1106,V,YoungstownState,24.0,29.0,53.0,144,154.5,700,79,2018-19
...,...,...,...,...,...,...,...,...,...,...,...
44769,1225,N,Pepperdine,25.0,41.0,66.0,153,152.5,220,78.5,2022-23
44770,1225,N,UtahState,42.0,40.0,82.0,3,3.5,-170,72.5,2022-23
44771,1225,N,WashingtonState,29.0,44.0,73.0,141,138.5,150,1.5,2022-23
44772,1225,V,SMU,31.0,26.0,57.0,4,134.5,170,70,2022-23


In [922]:
# drop rows containing strings to simplify calculation
def check_numeric(x): 
    try:
        float(x) or int(x)
        return True
    except ValueError:
        return False
prev_odds = prev_odds[prev_odds['ML'].apply(check_numeric)]
prev_odds = prev_odds[prev_odds['Close'].apply(check_numeric)]
prev_odds.ML = prev_odds.ML.astype(float)
prev_odds.Close = prev_odds.Close.astype(float)

In [923]:
prev_odds

,Date,VH,Team,1st,2nd,Final,Open,Close,ML,2H,Season
0,1106,V,Towson,19.0,23.0,42.0,130,129.0,6500.0,67,2018-19
1,1106,H,Virginia,28.0,45.0,73.0,25,26.0,-16500.0,11,2018-19
2,1106,V,WiscMilwaukee,37.0,16.0,53.0,146,147.5,1025.0,78,2018-19
3,1106,H,BostonCollege,27.0,46.0,73.0,17,16.0,-1550.0,12.5,2018-19
4,1106,V,YoungstownState,24.0,29.0,53.0,144,154.5,700.0,79,2018-19
...,...,...,...,...,...,...,...,...,...,...,...
44769,1225,N,Pepperdine,25.0,41.0,66.0,153,152.5,220.0,78.5,2022-23
44770,1225,N,UtahState,42.0,40.0,82.0,3,3.5,-170.0,72.5,2022-23
44771,1225,N,WashingtonState,29.0,44.0,73.0,141,138.5,150.0,1.5,2022-23
44772,1225,V,SMU,31.0,26.0,57.0,4,134.5,170.0,70,2022-23


In [1005]:
# reduce rows into half, split into home and away team  
away = prev_odds.iloc[::2, :].copy()
home = prev_odds.iloc[1::2, :].copy()

# drop repeated columns
home = home.drop(['Season'], axis=1) 
away = away.drop(['Date'], axis=1)

# change column names and reset index 
home_col = ['Date', 'VH1', 'homeTeam', 'home1st', 'home2nd', 'homeFinal', 'diffOpen', 'diffClose',
       'homeML', 'home2H']
away_col = ['VH2', 'awayTeam', 'away1st', 'away2nd', 'awayFinal', 'totalOpen', 'totalClose',
       'awayML', 'away2H', 'Season']
home.columns = home_col 
away.columns = away_col 
home.reset_index(inplace=True)
away.reset_index(inplace=True)
home.drop(['index'], axis=1, inplace=True)
away.drop(['index'], axis=1, inplace=True)


In [1006]:
# Concatenate into one dataframe
odds_df = pd.concat([home, away], axis=1)
odds_df = odds_df.dropna()
odds_df

,Date,VH1,homeTeam,home1st,home2nd,homeFinal,diffOpen,diffClose,homeML,home2H,VH2,awayTeam,away1st,away2nd,awayFinal,totalOpen,totalClose,awayML,away2H,Season
0,1106,H,Virginia,28.0,45.0,73.0,25,26,-16500,11,V,Towson,19.0,23.0,42.0,130,129,6500,67,2018-19
1,1106,H,BostonCollege,27.0,46.0,73.0,17,16,-1550,12.5,V,WiscMilwaukee,37.0,16.0,53.0,146,147.5,1025,78,2018-19
2,1106,H,Pittsburgh,27.0,42.0,69.0,14,13.5,-1100,7.5,V,YoungstownState,24.0,29.0,53.0,144,154.5,700,79,2018-19
3,1106,H,NotreDame,46.0,38.0,84.0,16,11.5,-750,78,V,IllinoisChicago,23.0,44.0,67.0,148,147,525,0.5,2018-19
4,1106,H,WesternMichigan,45.0,44.0,89.0,12.5,11.5,-750,2.5,V,DetroitU,28.0,48.0,76.0,153,153.5,525,80.5,2018-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22382,1225,H,Creighton,39.0,41.0,80.0,16,15.5,-1400,6.5,V,Depaul,28.0,37.0,65.0,147,145,800,77,2022-23
22383,1225,N,GeoWashington,24.0,43.0,67.0,146,144.5,125,4,N,SeattleU,41.0,44.0,85.0,2,2.5,-145,76,2022-23
22384,1225,N,Pepperdine,25.0,41.0,66.0,153,152.5,220,78.5,N,Iona,35.0,41.0,76.0,7,6,-260,0.5,2022-23
22385,1225,N,WashingtonState,29.0,44.0,73.0,141,138.5,150,1.5,N,UtahState,42.0,40.0,82.0,3,3.5,-170,72.5,2022-23


In [1046]:
# add homeFavored col to indicate whether the home team is favored 
def check_favorite(homeML, awayML): 
    return homeML < awayML 
odds_df['homeFavored'] = odds_df.apply(lambda x: check_favorite(x.homeML, x.awayML), axis=1)

In [1050]:
# switch diff and total cols where home is not favored 
temp_col = odds_df.totalClose.copy()
odds_df.loc[~odds_df['homeFavored'], 'totalClose'] = odds_df['diffClose'] 
odds_df.loc[~odds_df['homeFavored'], 'diffClose'] = temp_col
odds_df[~odds_df['homeFavored']][['diffClose', 'totalClose']]

,diffClose,totalClose
15,-15.5,158.5
30,-10.0,158.0
36,-7.5,146.5
39,-1.5,141.5
41,-4.0,152.5
...,...,...
22375,-8.0,148.0
22378,-2.0,127.0
22383,-2.5,144.5
22384,-6.0,152.5


In [1051]:
odds_df[['diffClose', 'totalClose']]

,diffClose,totalClose
0,26.0,129.0
1,16.0,147.5
2,13.5,154.5
3,11.5,147.0
4,11.5,153.5
...,...,...
22382,15.5,145.0
22383,-2.5,144.5
22384,-6.0,152.5
22385,-3.5,138.5


## Match TeamId

In [1011]:
sql = '''
SELECT DISTINCT teamMarket, teamId
FROM `cbbanalytics.models.gs__team_info` 
WHERE gender = 'MALE'
'''

In [1012]:
team_info = client.query(sql).to_dataframe()

In [965]:
# remove space in team names
team_info['teamMarket'] = team_info['teamMarket'].apply(lambda x: x.replace(' ', '').strip())
team_info = team_info.dropna()
team_info.head()

,teamMarket,teamId
0,Emmanuel(GA),103580
1,TennesseeTech,104314
2,MoreheadSt.,103947
3,Elmhurst,103574
4,NorfolkSt.,103994


In [762]:
# save teamId_info as excel file 
#team_info.to_excel('teamId_lookup.xlsx') 

In [1014]:
# create team info dictionary to match id 
team_info_dict = dict(zip(team_info.teamMarket, team_info.teamId))

In [1015]:
def match_teamId(x): 
    if x in team_info_dict: 
        return team_info_dict[x]

In [1016]:
match_teamId('Virginia')

104405

In [1017]:
# get unique teams and match id based on names
unique_teams = prev_odds.Team.unique()
unique_teams = pd.DataFrame(unique_teams, columns=['team'])
unique_id = unique_teams['team'].apply(match_teamId)
unique_teams['id'] = unique_id
print(unique_id)
print('Number of nulls: ', unique_id.isnull().sum())
unique_teams

0      104335.0
1      104405.0
2           NaN
3      103363.0
4           NaN
         ...   
412         NaN
413    104297.0
414         NaN
415         NaN
416         NaN
Name: team, Length: 417, dtype: float64
Number of nulls:  206


,team,id
0,Towson,104335.0
1,Virginia,104405.0
2,WiscMilwaukee,NaN
3,BostonCollege,103363.0
4,YoungstownState,NaN
...,...,...
412,QueensNC,NaN
413,Stonehill,104297.0
414,TexA&MCommerce,NaN
415,NaN,NaN


In [1018]:
# check # of null in unique teams
unique_teams.dropna(subset=["team"], inplace=True)
unique_teams.isnull().sum()

team      0
id      205
dtype: int64

In [1019]:
# use string manipulation to match id 
null_teams = unique_teams[unique_teams['id'].isnull()]
x = []
for i in range(len(unique_teams['team'])): 
    team_name = unique_teams['team'].iloc[i][-5: ]
    if unique_teams['team'].iloc[i][-5: ] == 'State' and np.isnan(unique_teams['id'].iloc[i]): 
        team_name = unique_teams['team'].iloc[i][:-5]
        if team_name in team_info_dict: 
            unique_teams['id'].iloc[i] = team_info_dict[team_name] 
    elif unique_teams['team'].iloc[i][-1: ] == 'U' and np.isnan(unique_teams['id'].iloc[i]): 
        team_name = unique_teams['team'].iloc[i][:-1]
        if team_name in team_info_dict: 
            unique_teams['id'].iloc[i] = team_info_dict[team_name] 
    elif unique_teams['team'].iloc[i][-3: ] == 'St.' and np.isnan(unique_teams['id'].iloc[i]):   
        team_name = unique_teams['team'].iloc[i][:-3]
        if team_name in team_info_dict: 
            unique_teams['id'].iloc[i] = team_info_dict[team_name] 
    elif unique_teams['team'].iloc[i][-2: ] == 'St' and np.isnan(unique_teams['id'].iloc[i]):   
        team_name = unique_teams['team'].iloc[i][:-2]
        if team_name in team_info_dict: 
            unique_teams['id'].iloc[i] = team_info_dict[team_name] 
print(unique_teams)
unique_teams.isnull().sum()

                 team        id
0              Towson  104335.0
1            Virginia  104405.0
2       WiscMilwaukee       NaN
3       BostonCollege  103363.0
4     YoungstownState       NaN
..                ...       ...
411          UtahTech  103539.0
412          QueensNC       NaN
413         Stonehill  104297.0
414    TexA&MCommerce       NaN
416  TexasA&MCommerce       NaN

[416 rows x 2 columns]


/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


team      0
id      156
dtype: int64

In [769]:
# save unique teams as excel file 
# unique_teams.to_excel('raw_teams.xlsx') 

In [1020]:
# manually matched id of the rest of the teams 
raw_teams_filled = pd.read_excel('data/raw_teams_filled.xlsx', index_col=0)
raw_teams_filled = raw_teams_filled.fillna(0)
raw_teams_filled['id'] = raw_teams_filled['id'].astype('int')
print(raw_teams_filled)
print(raw_teams_filled.isnull().sum())

                     team      id
0                Virginia  104405
1           BostonCollege  103363
2              Pittsburgh  104099
3               NotreDame  104026
4         WesternMichigan  104454
..                    ...     ...
413  NorthCarolinaCentral  103963
414      UTRioGrandValley  104138
415       FloridaAtlantic  103609
416      TexasA&MCommerce  104316
417           ULLafayette  103828

[417 rows x 2 columns]
team    0
id      0
dtype: int64


In [1021]:
# create team id look up dictionary 
id_lookup = dict(zip(raw_teams_filled.team, raw_teams_filled.id))

In [1022]:
# match id to odds dataframe
def teamId_lookup(x): 
    if x in id_lookup: 
        return id_lookup[x]
odds_df['homeId'] = odds_df['homeTeam'].apply(teamId_lookup)
odds_df['awayId'] = odds_df['awayTeam'].apply(teamId_lookup) 

In [1023]:
odds_df[['homeId', 'awayId']]

,homeId,awayId
0,104405,104335
1,103363,103917
2,104099,104515
3,104026,104362
4,104454,103536
...,...,...
22382,103510,103525
22383,103642,104209
22384,104090,103733
22385,104424,104389


In [1024]:
odds_df.head()

,Date,VH1,homeTeam,home1st,home2nd,homeFinal,diffOpen,diffClose,homeML,home2H,...,away2nd,awayFinal,totalOpen,totalClose,awayML,away2H,Season,homeFavored,homeId,awayId
0,1106,H,Virginia,28.0,45.0,73.0,25,26.0,-16500.0,11,...,23.0,42.0,130,129.0,6500.0,67,2018-19,True,104405,104335
1,1106,H,BostonCollege,27.0,46.0,73.0,17,16.0,-1550.0,12.5,...,16.0,53.0,146,147.5,1025.0,78,2018-19,True,103363,103917
2,1106,H,Pittsburgh,27.0,42.0,69.0,14,13.5,-1100.0,7.5,...,29.0,53.0,144,154.5,700.0,79,2018-19,True,104099,104515
3,1106,H,NotreDame,46.0,38.0,84.0,16,11.5,-750.0,78,...,44.0,67.0,148,147.0,525.0,0.5,2018-19,True,104026,104362
4,1106,H,WesternMichigan,45.0,44.0,89.0,12.5,11.5,-750.0,2.5,...,48.0,76.0,153,153.5,525.0,80.5,2018-19,True,104454,103536


In [1025]:
odds_df[odds_df['homeId'] == 0]['homeTeam'].unique()

array([], dtype=object)

In [1026]:
odds_df[odds_df['awayId'] == 0]['awayTeam'].unique()

array([], dtype=object)

## Clean odds table

In [1027]:
odds_df['diffClose'] = odds_df['diffClose'].fillna(0)
odds_df['totalClose'] = odds_df['totalClose'].fillna(0)

In [1053]:
# check for strings 
print(len(odds_df[~odds_df['diffClose'].apply(check_numeric)]['diffClose']))
print(len(odds_df[~odds_df['totalClose'].apply(check_numeric)]['totalClose']))
print(odds_df[~odds_df['diffClose'].apply(check_numeric)]['diffClose'].unique())

0
0
[]


In [1054]:
# apply function to both columns 
def check_diff_total(diff, total): 
    if diff not in str_list and total not in str_list: 
        return diff > 80 and total < 80
    return False 
switch_filter = odds_df.apply(lambda x: check_diff_total(x.diffClose, x.totalClose), axis=1)
odds_df[switch_filter][['diffClose', 'totalClose']]

,diffClose,totalClose


In [1056]:
# switch wrong columns
temp_col = odds_df.totalClose.copy()
odds_df.loc[switch_filter, 'totalClose'] = odds_df['diffClose'] 
odds_df.loc[switch_filter, 'diffClose'] = temp_col
odds_df[switch_filter][['diffClose', 'totalClose']]

,diffClose,totalClose


In [1057]:
odds_df[['diffClose', 'totalClose']]

,diffClose,totalClose
0,26.0,129.0
1,16.0,147.5
2,13.5,154.5
3,11.5,147.0
4,11.5,153.5
...,...,...
22382,15.5,145.0
22383,-2.5,144.5
22384,-6.0,152.5
22385,-3.5,138.5


In [1034]:
odds_df[['diffClose', 'homeML', 'awayML']]

,diffClose,homeML,awayML
0,26.0,-16500.0,6500.0
1,16.0,-1550.0,1025.0
2,13.5,-1100.0,700.0
3,11.5,-750.0,525.0
4,11.5,-750.0,525.0
...,...,...,...
22382,15.5,-1400.0,800.0
22383,2.5,125.0,-145.0
22384,6.0,220.0,-260.0
22385,3.5,150.0,-170.0


In [1035]:
# drop irrelevant columns 
odds_df = odds_df.drop(['VH1', 'home1st', 'home2nd', 'diffOpen', 'home2H', 'VH2', 'away1st', 'away2nd', 'totalOpen', 'away2H'], axis=1)

In [1036]:
# correct the points difference to negative when home is not favored 
odds_df.loc[~odds_df['homeFavored'], 'diffClose'] *= -1
odds_df

,Date,homeTeam,homeFinal,diffClose,homeML,awayTeam,awayFinal,totalClose,awayML,Season,homeFavored,homeId,awayId
0,1106,Virginia,73.0,26.0,-16500.0,Towson,42.0,129.0,6500.0,2018-19,True,104405,104335
1,1106,BostonCollege,73.0,16.0,-1550.0,WiscMilwaukee,53.0,147.5,1025.0,2018-19,True,103363,103917
2,1106,Pittsburgh,69.0,13.5,-1100.0,YoungstownState,53.0,154.5,700.0,2018-19,True,104099,104515
3,1106,NotreDame,84.0,11.5,-750.0,IllinoisChicago,67.0,147.0,525.0,2018-19,True,104026,104362
4,1106,WesternMichigan,89.0,11.5,-750.0,DetroitU,76.0,153.5,525.0,2018-19,True,104454,103536
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22382,1225,Creighton,80.0,15.5,-1400.0,Depaul,65.0,145.0,800.0,2022-23,True,103510,103525
22383,1225,GeoWashington,67.0,-2.5,125.0,SeattleU,85.0,144.5,-145.0,2022-23,False,103642,104209
22384,1225,Pepperdine,66.0,-6.0,220.0,Iona,76.0,152.5,-260.0,2022-23,False,104090,103733
22385,1225,WashingtonState,73.0,-3.5,150.0,UtahState,82.0,138.5,-170.0,2022-23,False,104424,104389


In [1037]:
# convert Date into datetime to match format in sql data 
def convert_datetime(date, season): 
    date = str(date)
    month = int(date[:-2])
    if month > 6: 
        year = '20' + season[2:4] 
    else: 
        year = '20' + season[-2:] 
    if len(date) < 4: 
        date = str(0) + date
    date = year + date 
    datetime_object = datetime.strptime(date, "%Y%m%d")
    date_string = datetime_object.strftime("%Y-%m-%d")
    return date_string
odds_df.Date = odds_df.apply(lambda x: convert_datetime(x.Date, x.Season), axis=1)
odds_df

,Date,homeTeam,homeFinal,diffClose,homeML,awayTeam,awayFinal,totalClose,awayML,Season,homeFavored,homeId,awayId
0,2018-11-06,Virginia,73.0,26.0,-16500.0,Towson,42.0,129.0,6500.0,2018-19,True,104405,104335
1,2018-11-06,BostonCollege,73.0,16.0,-1550.0,WiscMilwaukee,53.0,147.5,1025.0,2018-19,True,103363,103917
2,2018-11-06,Pittsburgh,69.0,13.5,-1100.0,YoungstownState,53.0,154.5,700.0,2018-19,True,104099,104515
3,2018-11-06,NotreDame,84.0,11.5,-750.0,IllinoisChicago,67.0,147.0,525.0,2018-19,True,104026,104362
4,2018-11-06,WesternMichigan,89.0,11.5,-750.0,DetroitU,76.0,153.5,525.0,2018-19,True,104454,103536
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22382,2022-12-25,Creighton,80.0,15.5,-1400.0,Depaul,65.0,145.0,800.0,2022-23,True,103510,103525
22383,2022-12-25,GeoWashington,67.0,-2.5,125.0,SeattleU,85.0,144.5,-145.0,2022-23,False,103642,104209
22384,2022-12-25,Pepperdine,66.0,-6.0,220.0,Iona,76.0,152.5,-260.0,2022-23,False,104090,103733
22385,2022-12-25,WashingtonState,73.0,-3.5,150.0,UtahState,82.0,138.5,-170.0,2022-23,False,104424,104389


In [1040]:
# odds_df.to_csv('historical_betting_odds.csv')

In [1039]:
odds_df.isna().sum()

Date           0
homeTeam       0
homeFinal      0
diffClose      0
homeML         0
awayTeam       0
awayFinal      0
totalClose     0
awayML         0
Season         0
homeFavored    0
homeId         0
awayId         0
dtype: int64